# Data cleaning

## Première étape : Importer et Visualiser l'entête des données

L'importance de comprendre le dataset avec un visuel succint permet de deviner quelles seront les modification à apporter à notre dataset pour le nettoyer.

Dans notre première étape nous allons importer pandas, qui est un framework python permettant d'importer des datasets sous forme de Dataframe.
Numpy est une bibliothèque utilisée par pandas qui gère des tableaux/matrices, il me servira à modifier des tableaux si nécessaires.

In [3]:
import pandas as pd
import numpy as np
from pprint import pprint


## Remarque 

Le dataset contient ici  des json pour dataset. Il nous faudra trouver un moyen de convertir ces json en colonnes qui seront plus facile à traiter.

Les colonnes à multiplier sont :

device ; geoNetwork ; totals ; traficSource

In [4]:
import os
import json
from pandas.io.json import json_normalize

def load_df(csv_path='./Dataset/train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df


In [5]:
data = load_df(nrows = 30000)

data.head()

Loaded train.csv. Shape: (30000, 54)


,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserSize,...,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,Chrome,not available in demo dataset,...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,Firefox,not available in demo dataset,...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386,Chrome,not available in demo dataset,...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
3,Organic Search,20160902,4763447161404445595,4763447161404445595_1472881213,Not Socially Engaged,1472881213,1,1472881213,UC Browser,not available in demo dataset,...,NaN,NaN,NaN,NaN,(not set),NaN,google + online,organic,NaN,google
4,Organic Search,20160902,27294437909732085,27294437909732085_1472822600,Not Socially Engaged,1472822600,2,1472822600,Chrome,not available in demo dataset,...,NaN,NaN,NaN,NaN,(not set),True,(not provided),organic,NaN,google


## Nettoyage

Il semble y avoir beaucoup de NaN dans ces colonnes nouvellement créées. Pour ce, nous devons afficher les ratios de NaN présents dans chaques colonnes. Si le taux de NaN est supérieur à 80% ou supérieur à 0 on étudiera plus longuement cette question.

In [6]:
listnan80 = []
listnan = []
for i in data:
    if data[i].isna().sum() > len(data[i])*0.8:
        listnan80.append((i,data[i].isna().sum(),len(data[i])))
    elif data[i].isna().sum() > 0 :
        listnan.append((i,data[i].isna().sum(),len(data[i])))
pprint(listnan80)
pprint(listnan)

[('totals.transactionRevenue', 29684, 30000),
 ('trafficSource.adContent', 29680, 30000),
 ('trafficSource.adwordsClickInfo.adNetworkType', 29356, 30000),
 ('trafficSource.adwordsClickInfo.gclId', 29336, 30000),
 ('trafficSource.adwordsClickInfo.isVideoAd', 29356, 30000),
 ('trafficSource.adwordsClickInfo.page', 29356, 30000),
 ('trafficSource.adwordsClickInfo.slot', 29356, 30000)]
[('totals.bounces', 14466, 30000),
 ('totals.newVisits', 6329, 30000),
 ('totals.pageviews', 2, 30000),
 ('trafficSource.isTrueDirect', 21594, 30000),
 ('trafficSource.keyword', 17477, 30000),
 ('trafficSource.referralPath', 17684, 30000)]


#### Analyse

Apparament les colonnes adContent, adNetworkType, isVideoAd, page, slot et transactionRevenue posent problème dans la quantité de data qu'elles présentent.

Mon humble avis me dit qu'un NaN peut correspondre à un True, un False ou un donnée non récupérée.
Comment savoir ?
Il n'y a malheureusement aucune manière de le savoir à moins d'avoir un contact direct avec la personne qui a généré ce data.

On peut néanmoins analyser ces colonnes vides pour voir le nombre de valeurs différentes présentes dans ces colonnes.

In [7]:
tmpd = []
for a in listnan80:
    tmpd.append((a[0],data[a[0]].nunique()))
print("Les >80% : \n")
for i in tmpd:
    print(i)

print("\n\n les >0 : \n")
tmpd2 = []
for a in listnan:
    tmpd2.append((a[0],data[a[0]].nunique()))
for i in tmpd2:
    print(i)

Les >80% : 

('totals.transactionRevenue', 273)
('trafficSource.adContent', 25)
('trafficSource.adwordsClickInfo.adNetworkType', 1)
('trafficSource.adwordsClickInfo.gclId', 603)
('trafficSource.adwordsClickInfo.isVideoAd', 1)
('trafficSource.adwordsClickInfo.page', 4)
('trafficSource.adwordsClickInfo.slot', 2)


 les >0 : 

('totals.bounces', 1)
('totals.newVisits', 1)
('totals.pageviews', 93)
('trafficSource.isTrueDirect', 1)
('trafficSource.keyword', 240)
('trafficSource.referralPath', 267)


On peut clairement deviner que adnetworkType et isVideoAd peuvent avoir une confusion entre le NaN et le False/True
le reste des colonnes peut être supprimé.
Quand aux colonnes contenant quelques NaN certaines peuvent être traitées pour compenser ces valeurs. L'étape suivante est de lire le data et comprendre comment remplir ces cases vides.

In [24]:
data["trafficSource.adwordsClickInfo.adNetworkType"].any() # Comme la valeur est textuelle on ne peut pas la remplir

data["trafficSource.adwordsClickInfo.isVideoAd"].any()  #Comme la valeur est booléenne on pourrait remplir le reste en tant que true

False

In [38]:
# Pour ces exemples, le reste est complété par les valeurs opposées
data["totals.newVisits"].fillna(0)
data["totals.bounces"].fillna(0)
data["trafficSource.isTrueDirect"].fillna(False)

# Pour les dernières colonnes, une catégorie déjà existante sera utilisée du fait de leur 

data["totals.pageviews"].fillna(0)
data["trafficSource.keyword"].fillna("(not provided)")
data["trafficSource.referralPath"].fillna("/")



array([nan, '/', '/corp/google.com/study/incentives/working-with-perks',
       '/od/Things-To-Do-in-Silicon-Valley/fl/How-To-Visit-the-Googleplex-the-Google-Head-Office-in-Mountain-View.htm',
       '/a/google.com/google-merchandise-store/on-site-store',
       '/analytics/web/', '/intl/en/permissions/using-the-logo.html',
       '/permissions/using-the-logo.html',
       '/YKEI_mrn/items/c10b14f9a69ff71b1b7a',
       '/analytics/web/inpage_launch',
       '/intl/de/permissions/using-the-logo.html',
       '/intl/pl/permissions/using-the-logo.html',
       '/a/google.com/google-merchandise-store/on-site-store/mountain-view-store',
       '/intl/ALL/permissions/using-the-logo.html',
       '/_/scs/mail-static/_/js/k=gmail.main.en.VSE1BHcxoTk.O/m=m_i,t,it/am=OotHDjBPxv3BuIZhQFf6Kt35738-vVR-6Oce1Z8IiCKvAv1v9v8A_g_kp20U/rt=h/d=1/rs=AHGWq9CcYnkIxVeeMRcVhXaI7V_8FR2FlQ',
       '/hangouts/_/cdjako2b4rb7vhgj5g74n2mytqe',
       '/google-merchandise-store', '/optimize/home/',
       '/deal/-ds

Le nettoyage est donc terminé !

Nous allons donc procéder à étudier le data dans son ensemble et chercher une question!